In [8]:
from bs4 import BeautifulSoup
import urllib.request as req
import pandas as pd
import datetime
def getweather(address):

    today = datetime.datetime.today().strftime('%Y%m%d')
    tomorrow = (datetime.datetime.today() + datetime.timedelta(days=1)).strftime('%Y%m%d')
    # print(today)
    # print(tomorrow)

    dicloc = {
        'address': ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도',
                    '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주도'],
        'x': [60, 98, 89, 55, 58, 67, 102, 66, 60, 73, 69, 68, 63, 51, 89, 91, 52],
        'y': [127, 76, 90, 124, 74, 100, 84, 103, 120, 134, 107, 100, 89, 67, 91, 77, 38]}

    dfloc = pd.DataFrame(dicloc)
    xyloc = dfloc[dfloc['address'].isin([address])]

    # api = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getUltraSrtFcst?serviceKey=' #초단기예보
    api = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?serviceKey='  # 동네예보
    key = '6TuB1szn0aNUEln7nrp6jgTQuv5WvoOgxRfPqdBtNPKTVIwbPV7SGmirrRyUIU95CP8oFZHn2f2Yr3zOWPiSdA%3D%3D'
    numOfRows = '1000'
    pageNo = '1'
    base_date = today
    base_time = '0200'
    # print(xyloc.iloc[0][1])
    # print(xyloc.iloc[0][2])

    nx = xyloc.iloc[0][1].astype('str')
    ny = xyloc.iloc[0][2].astype('str')

    savename = 'tmpweather.xml'

    url = api + key + '&numOfRows=' + \
          numOfRows + '&pageNo=' + pageNo + '&base_date=' + base_date + '&base_time=' + base_time + '&nx=' + nx + '&ny=' + ny

    req.urlretrieve(url, savename)

    xml = open(savename, 'r', encoding='utf-8').read()
    soup = BeautifulSoup(xml, 'xml')

    # print(soup.find_all('item'))

    weatherList = []
    for itemList in soup.find_all('item'):
        baseDate = itemList.find('baseDate').string
        baseTime = itemList.find('baseTime').string
        category = itemList.find('category').string
        fcstDate = itemList.find('fcstDate').string
        fcstTime = itemList.find('fcstTime').string
        fcstValue = itemList.find('fcstValue').string
        nx = itemList.find('nx').string
        ny = itemList.find('ny').string
        weatherList.append([baseDate, baseTime, category, fcstDate, fcstTime, fcstValue, nx, ny])

    df = pd.DataFrame(weatherList,
                      columns=['baseDate', 'baseTime', 'category', 'fcstDate', 'fcstTime', 'fcstValue', 'nx', 'ny'])

    df['ffcstValue'] = df['fcstValue'].astype('float')
    df_twoday = df[df['fcstDate'].isin([today, tomorrow])].groupby(['fcstDate', 'category']).mean(
        'fcstValue').reset_index()
    pv_towday = df_twoday.pivot(index=['fcstDate'], columns='category', values='ffcstValue').reset_index()
    pv_towday['temp_avg'] = pv_towday[['TMN', 'TMX']].mean(axis=1)
    pv_towday[['fcstDate', 'R06', 'REH', 'temp_avg']]

    returndic = {'today_rain': round(pv_towday[pv_towday['fcstDate'] == today].R06.values[0],1),
                 'today_humidity': round(pv_towday[pv_towday['fcstDate'] == today].REH.values[0],1),
                 'today_temp': round(pv_towday[pv_towday['fcstDate'] == today].temp_avg.values[0],1),
                 'today_ws': round(pv_towday[pv_towday['fcstDate'] == today].WSD.values[0],1),
                 'tomday_rain': round(pv_towday[pv_towday['fcstDate'] == tomorrow].R06.values[0],1),
                 'tomday_humidity': round(pv_towday[pv_towday['fcstDate'] == tomorrow].REH.values[0],1),
                 'tomday_temp': round(pv_towday[pv_towday['fcstDate'] == tomorrow].temp_avg.values[0],1),
                 'tomday_ws': round(pv_towday[pv_towday['fcstDate'] == tomorrow].WSD.values[0],1),
                 'address': address
                 }
    return returndic

returndic = getweather('서울특별시')

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [55]:
import pymysql
import pandas as pd

conn = pymysql.connect(host='multi-bigdata.cljkqcsbb9ok.ap-northeast-2.rds.amazonaws.com',port=3306,user='edu02',passwd='edu02', db='edudb01', cursorclass=pymysql.cursors.DictCursor)
try:
    cur = conn.cursor()
    sql = '''
        SELECT *
        FROM Rsquare
        WHERE 1 = 1
        
    '''
#    sqlw = ' AND city IN (' + '서울특별시' +')'
    
    cur.execute(sql)
    result = cur.fetchall()
finally:
    conn.close()
print('마리아에서 불러온 행 수 :', len(result))

dfProduct = pd.DataFrame(result)

마리아에서 불러온 행 수 : 4894


In [56]:
display(dfProduct)
display(returndic)



,city,rsquare,response,xa,b,dvlist,grouprow,shiftrow
0,서울특별시,-0.012990,가구/인테리어,"[0.03182761547281937, -0.007998121328835995, -...",[72.44171015],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0
1,서울특별시,0.087818,가구/인테리어 > DIY자재/용품,"[0.2712539761764443, -0.012031723854340842, 2....",[70.52208832],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0
2,서울특별시,0.189701,가구/인테리어 > DIY자재/용품 > 가구부속품,"[0.46907048232217285, -0.017049331220885745, 2...",[64.79812436],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0
3,서울특별시,0.207554,가구/인테리어 > DIY자재/용품 > 데코스티커,"[-0.19242629454500268, -0.012482480193632387, ...",[69.07066997],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0
4,서울특별시,0.202003,가구/인테리어 > DIY자재/용품 > 목재,"[0.2944483538472975, -0.020887708844167935, 3....",[71.67481585],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0
...,...,...,...,...,...,...,...,...
4889,강원도,-0.016961,화장품/미용 > 헤어케어 > 트리트먼트,"[0.015192576572405681, -0.05607041259620522, 2...",[67.01035735],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0
4890,강원도,-0.155873,화장품/미용 > 헤어케어 > 헤어미스트,"[0.43033386830118314, 0.008556814138243652, 0....",[29.40127503],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0
4891,강원도,-0.056329,화장품/미용 > 헤어케어 > 헤어에센스,"[0.16177716565378422, -0.15318827248134914, 1....",[67.92049897],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0
4892,강원도,0.199285,화장품/미용 > 헤어케어 > 헤어케어세트,"[0.8709021977117957, -0.016113217574568583, 1....",[35.20425133],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0


{'today_rain': 0.0,
 'today_humidity': 70.0,
 'today_temp': 12.5,
 'today_ws': 1.6,
 'tomday_rain': 0.0,
 'tomday_humidity': 74.4,
 'tomday_temp': 13.0,
 'tomday_ws': 1.3,
 'address': '서울특별시'}

In [53]:
addWeather = pd.DataFrame()
for add in ['서울특별시', '강원도']:
    addWeather = addWeather.append(pd.DataFrame(getweather(add), index=[0]), ignore_index=True)
addWeather

,today_rain,today_humidity,today_temp,today_ws,tomday_rain,tomday_humidity,tomday_temp,tomday_ws,address
0,0.0,70.0,12.5,1.6,0.0,74.4,13.0,1.3,서울특별시
1,0.0,73.3,10.0,1.2,0.0,79.4,11.5,0.8,강원도


In [127]:
alldf = pd.merge(dfProduct, addWeather, left_on=["city"], right_on=["address"], how="inner")
alldf

,city,rsquare,response,xa,b,dvlist,grouprow,shiftrow,today_rain,today_humidity,today_temp,today_ws,tomday_rain,tomday_humidity,tomday_temp,tomday_ws,address
0,서울특별시,-0.012990,가구/인테리어,"[0.03182761547281937, -0.007998121328835995, -...",[72.44171015],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,70.0,12.5,1.6,0.0,74.4,13.0,1.3,서울특별시
1,서울특별시,0.087818,가구/인테리어 > DIY자재/용품,"[0.2712539761764443, -0.012031723854340842, 2....",[70.52208832],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,70.0,12.5,1.6,0.0,74.4,13.0,1.3,서울특별시
2,서울특별시,0.189701,가구/인테리어 > DIY자재/용품 > 가구부속품,"[0.46907048232217285, -0.017049331220885745, 2...",[64.79812436],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,70.0,12.5,1.6,0.0,74.4,13.0,1.3,서울특별시
3,서울특별시,0.207554,가구/인테리어 > DIY자재/용품 > 데코스티커,"[-0.19242629454500268, -0.012482480193632387, ...",[69.07066997],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,70.0,12.5,1.6,0.0,74.4,13.0,1.3,서울특별시
4,서울특별시,0.202003,가구/인테리어 > DIY자재/용품 > 목재,"[0.2944483538472975, -0.020887708844167935, 3....",[71.67481585],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,70.0,12.5,1.6,0.0,74.4,13.0,1.3,서울특별시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4889,강원도,-0.016961,화장품/미용 > 헤어케어 > 트리트먼트,"[0.015192576572405681, -0.05607041259620522, 2...",[67.01035735],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,73.3,10.0,1.2,0.0,79.4,11.5,0.8,강원도
4890,강원도,-0.155873,화장품/미용 > 헤어케어 > 헤어미스트,"[0.43033386830118314, 0.008556814138243652, 0....",[29.40127503],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,73.3,10.0,1.2,0.0,79.4,11.5,0.8,강원도
4891,강원도,-0.056329,화장품/미용 > 헤어케어 > 헤어에센스,"[0.16177716565378422, -0.15318827248134914, 1....",[67.92049897],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,73.3,10.0,1.2,0.0,79.4,11.5,0.8,강원도
4892,강원도,0.199285,화장품/미용 > 헤어케어 > 헤어케어세트,"[0.8709021977117957, -0.016113217574568583, 1....",[35.20425133],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,73.3,10.0,1.2,0.0,79.4,11.5,0.8,강원도


In [153]:
alldf['xa'].to_list()[0].replace('[', '').replace(']','').split(' ')
# '[0.03182761547281937, -0.007998121328835995, -0.03520869764110174, 0.03520869764110173]'
['temp_avg', 'amount_of_rain', 'wk0', 'wk1']
alldf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4894 entries, 0 to 4893
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city             4894 non-null   object 
 1   rsquare          4894 non-null   float64
 2   response         4894 non-null   object 
 3   xa               4894 non-null   object 
 4   b                4894 non-null   object 
 5   dvlist           4894 non-null   object 
 6   grouprow         4894 non-null   object 
 7   shiftrow         4894 non-null   int64  
 8   today_rain       4894 non-null   float64
 9   today_humidity   4894 non-null   float64
 10  today_temp       4894 non-null   float64
 11  today_ws         4894 non-null   float64
 12  tomday_rain      4894 non-null   float64
 13  tomday_humidity  4894 non-null   float64
 14  tomday_temp      4894 non-null   float64
 15  tomday_ws        4894 non-null   float64
 16  address          4894 non-null   object 
 17  xa2           

In [184]:
# weeksting과 bin을 만듬
today = datetime.datetime.today()
tomorrow = (datetime.datetime.today() + datetime.timedelta(days=1))

# 4:금 5:토 6:일 0:월 
def get_week(dt):
    cat = ''
    if pd.to_datetime(dt).weekday() == 5 : cat = 1
    elif pd.to_datetime(dt).weekday() == 6 : cat = 1
    elif pd.to_datetime(dt).weekday() == 4 : cat = 1
    elif pd.to_datetime(dt).weekday() == 0 : cat = 1        
    else : cat = 0
    return cat

# 주말처리 토일월과 그외 평일이면 wk0가 1 휴일이면 wk1이 1
if get_week(today) == 0 : # 평일이면
    alldf['towk0'] = 1
    alldf['towk1'] = 0
else :
    alldf['towk0'] = 0
    alldf['towk1'] = 1
    
if get_week(today) == 0 : # 평일이면
    alldf['tomwk0'] = 1
    alldf['tomwk1'] = 0
else :
    alldf['tomwk0'] = 0
    alldf['tomwk1'] = 1


alldf['xa2'] = alldf['xa'].apply(lambda x : x.replace('[', '').replace(']', '').split(','))
alldf['b2'] = alldf['b'].apply(lambda x : x.replace('[', '').replace(']', '').split(','))
alldf['estV'] = alldf.apply(lambda x : float(x['xa2'][0]) * x['tomday_temp'] + float(x['xa2'][1]) * x['tomday_rain'] + \
                                       float(x['xa2'][2]) * x['tomwk0'] + float(x['xa2'][3]) * x['tomwk1'] + float(x['b2'][0]), axis=1)

# display(alldf['xa2'])
# display(float(alldf['xa2'][0][0]))
# display(float(alldf['xa2'][0][3]))

In [86]:
alldf['xa'][0].replace('[', '').replace(']','').split(',')[0].astype('float')
pdf_Sigungu_dist['udistance'] = pdf_Sigungu_dist.apply(lambda x : distance.euclidean((x['sigungu_x'], x['sigungu_y']), (x['senser_x'], x['senser_y'])), axis=1)

AttributeError: 'str' object has no attribute 'astype'

In [98]:

alldf['b'][0].replace('[', '').replace(']', '').split(',')

AttributeError: 'list' object has no attribute 'astype'

In [187]:
alldf

,city,rsquare,response,xa,b,dvlist,grouprow,shiftrow,today_rain,today_humidity,...,address,xa2,estV,towk_bin,tomwk_bin,towk0,towk1,tomwk0,tomwk1,b2
0,서울특별시,-0.012990,가구/인테리어,"[0.03182761547281937, -0.007998121328835995, -...",[72.44171015],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,70.0,...,서울특별시,"[0.03182761547281937, -0.007998121328835995, ...",72.890678,1,1,0,1,0,1,[72.44171015]
1,서울특별시,0.087818,가구/인테리어 > DIY자재/용품,"[0.2712539761764443, -0.012031723854340842, 2....",[70.52208832],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,70.0,...,서울특별시,"[0.2712539761764443, -0.012031723854340842, ...",71.176183,1,1,0,1,0,1,[70.52208832]
2,서울특별시,0.189701,가구/인테리어 > DIY자재/용품 > 가구부속품,"[0.46907048232217285, -0.017049331220885745, 2...",[64.79812436],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,70.0,...,서울특별시,"[0.46907048232217285, -0.017049331220885745, ...",67.908298,1,1,0,1,0,1,[64.79812436]
3,서울특별시,0.207554,가구/인테리어 > DIY자재/용품 > 데코스티커,"[-0.19242629454500268, -0.012482480193632387, ...",[69.07066997],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,70.0,...,서울특별시,"[-0.19242629454500268, -0.012482480193632387,...",63.925632,1,1,0,1,0,1,[69.07066997]
4,서울특별시,0.202003,가구/인테리어 > DIY자재/용품 > 목재,"[0.2944483538472975, -0.020887708844167935, 3....",[71.67481585],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,70.0,...,서울특별시,"[0.2944483538472975, -0.020887708844167935, ...",71.718102,1,1,0,1,0,1,[71.67481585]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4889,강원도,-0.016961,화장품/미용 > 헤어케어 > 트리트먼트,"[0.015192576572405681, -0.05607041259620522, 2...",[67.01035735],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,73.3,...,강원도,"[0.015192576572405681, -0.05607041259620522, ...",65.108836,1,1,0,1,0,1,[67.01035735]
4890,강원도,-0.155873,화장품/미용 > 헤어케어 > 헤어미스트,"[0.43033386830118314, 0.008556814138243652, 0....",[29.40127503],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,73.3,...,강원도,"[0.43033386830118314, 0.008556814138243652, ...",33.571284,1,1,0,1,0,1,[29.40127503]
4891,강원도,-0.056329,화장품/미용 > 헤어케어 > 헤어에센스,"[0.16177716565378422, -0.15318827248134914, 1....",[67.92049897],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,73.3,...,강원도,"[0.16177716565378422, -0.15318827248134914, ...",68.000956,1,1,0,1,0,1,[67.92049897]
4892,강원도,0.199285,화장품/미용 > 헤어케어 > 헤어케어세트,"[0.8709021977117957, -0.016113217574568583, 1....",[35.20425133],"['temp_avg', 'amount_of_rain', 'wk0', 'wk1']",1D,0,0.0,73.3,...,강원도,"[0.8709021977117957, -0.016113217574568583, ...",43.620029,1,1,0,1,0,1,[35.20425133]


In [234]:

def getShopTop10Product(city='서울특별시') :
    # Rsquare를 추출
    conn = pymysql.connect(host='multi-bigdata.cljkqcsbb9ok.ap-northeast-2.rds.amazonaws.com', port=3306, user='edu02',
                           passwd='edu02', db='edudb01', cursorclass=pymysql.cursors.DictCursor)
    try:
        cur = conn.cursor()
        sql = '''
            SELECT *
            FROM Rsquare
            WHERE 1 = 1
        '''
        sql = sql + 'AND city = "' + city + '" ' 
        cur.execute(sql)
        result = cur.fetchall()
    finally:
        conn.close()
    dfProduct = pd.DataFrame(result)

    # 날씨를 추출
    addWeather = pd.DataFrame(getweather(add), index=[0])

    # 날씨와 Rsquare를 병합
    alldf = pd.merge(dfProduct, addWeather, left_on=["city"], right_on=["address"], how="inner")


    # weeksting과 bin을 만듬
    today = datetime.datetime.today()
    tomorrow = (datetime.datetime.today() + datetime.timedelta(days=1))


    # 주말처리 토일월과 그외 평일이면 wk0가 1 휴일이면 wk1이 1
    if get_week(today) == 0:  # 평일이면
        alldf['towk0'] = 1
        alldf['towk1'] = 0
    else:
        alldf['towk0'] = 0
        alldf['towk1'] = 1

    if get_week(today) == 0:  # 평일이면
        alldf['tomwk0'] = 1
        alldf['tomwk1'] = 0
    else:
        alldf['tomwk0'] = 0
        alldf['tomwk1'] = 1
    
    # 내일의 예상 클릭율
    alldf['xa2'] = alldf['xa'].apply(lambda x: x.replace('[', '').replace(']', '').split(','))
    alldf['b2'] = alldf['b'].apply(lambda x: x.replace('[', '').replace(']', '').split(','))
    
    # 내일의 예상 클릭율
    alldf['tomestV'] = alldf.apply(lambda x: float(x['xa2'][0]) * x['tomday_temp'] + float(x['xa2'][1]) * x['tomday_rain'] + \
                                          float(x['xa2'][2]) * x['tomwk0'] + float(x['xa2'][3]) * x['tomwk1'] + float(x['b2'][0]), axis=1)
    # 오늘의 예상 클릭율
    alldf['toestV'] = alldf.apply(lambda x: float(x['xa2'][0]) * x['today_temp'] + float(x['xa2'][1]) * x['today_rain'] + \
                                          float(x['xa2'][2]) * x['towk0'] + float(x['xa2'][3]) * x['towk1'] + float(x['b2'][0]), axis=1)

    alldf['incRate'] = alldf.apply(lambda x: (x['tomestV'] - x['toestV']) / x['toestV'], axis=1)
                                 
    returndf = alldf[['city', 'rsquare', 'response', 'toestV', 'tomestV', 'incRate']]

#     top10 = lambda x: x.sort_values(by=['rsquare', 'incRate'], axis=0, ascending=False)[:6]
#     resultdfbycity = returndf.groupby('city').apply(top10)

    resultdfbycity = returndf.sort_values(by=['rsquare', 'incRate'], axis=0, ascending=False)[:6]
    
    dicsum = resultdfbycity.to_dict('index')
    print(dicsum)
    for x in dicsum:
        print(x.city)
        
    return dicsum
    
    
print(getShopTop10Product('강원도'))

{234: {'city': '강원도', 'rsquare': 0.788394, 'response': '디지털/가전 > 네트워크장비 > 무선모뎀', 'toestV': 39.26993912212535, 'tomestV': 35.44018977377205, 'incRate': -0.0975236894674878}, 1385: {'city': '강원도', 'rsquare': 0.78339, 'response': '스포츠/레저 > 테니스 > 테니스의류', 'toestV': 31.104290999771116, 'tomestV': 34.46124288740709, 'incRate': 0.10792568419774219}, 1704: {'city': '강원도', 'rsquare': 0.781769, 'response': '출산/육아 > 위생/건강용품 > 기저귀휴지통', 'toestV': 36.161951655339095, 'tomestV': 39.71532986373442, 'incRate': 0.09826289914501027}, 2014: {'city': '강원도', 'rsquare': 0.778791, 'response': '패션잡화 > 선글라스/안경테 > 안경줄', 'toestV': 40.169938419405824, 'tomestV': 43.38191882241433, 'incRate': 0.0799598040074868}, 221: {'city': '강원도', 'rsquare': 0.778018, 'response': '디지털/가전 > 네트워크장비', 'toestV': 47.15756334615102, 'tomestV': 43.93376824625408, 'incRate': -0.06836220684756954}, 1961: {'city': '강원도', 'rsquare': 0.776304, 'response': '패션잡화 > 남성가방 > 힙색', 'toestV': 42.012889079933, 'tomestV': 45.46304881811181, 'incRate':

AttributeError: 'int' object has no attribute 'city'

## 7일씩 날씨 가져오기

In [240]:
def get_7weather(address):
    
    today_7 = (datetime.datetime.today() + datetime.timedelta(days=-7)).strftime('%Y%m%d')
    today = datetime.datetime.today().strftime('%Y%m%d')
    tomorrow = (datetime.datetime.today() + datetime.timedelta(days=1)).strftime('%Y%m%d')
    print(today_7)
    print(today)
    print(tomorrow)
    
    dicloc = {
        'address': ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도',
                    '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주도'],
        'x': [60, 98, 89, 55, 58, 67, 102, 66, 60, 73, 69, 68, 63, 51, 89, 91, 52],
        'y': [127, 76, 90, 124, 74, 100, 84, 103, 120, 134, 107, 100, 89, 67, 91, 77, 38]}

    dfloc = pd.DataFrame(dicloc)
    xyloc = dfloc[dfloc['address'].isin([address])]

    # api = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getUltraSrtFcst?serviceKey=' # 초단기예보
    api = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getUltraSrtNcst?serviceKey='   # 초간기실황
    # api = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?serviceKey='  # 동네예보
    key = '6TuB1szn0aNUEln7nrp6jgTQuv5WvoOgxRfPqdBtNPKTVIwbPV7SGmirrRyUIU95CP8oFZHn2f2Yr3zOWPiSdA%3D%3D'
    numOfRows = '1000'
    pageNo = '1'
    base_date = today_7
    base_time = '0200'
    # print(xyloc.iloc[0][1])
    # print(xyloc.iloc[0][2])

    nx = xyloc.iloc[0][1].astype('str')
    ny = xyloc.iloc[0][2].astype('str')

    savename = 'tmpweather.xml'

    url = api + key + '&numOfRows=' + \
          numOfRows + '&pageNo=' + pageNo + '&base_date=' + base_date + '&base_time=' + base_time + '&nx=' + nx + '&ny=' + ny
    print(url)
    req.urlretrieve(url, savename)

    xml = open(savename, 'r', encoding='utf-8').read()
    soup = BeautifulSoup(xml, 'xml')

    # print(soup.find_all('item'))

    weatherList = []
    for itemList in soup.find_all('item'):
        baseDate = itemList.find('baseDate').string
        baseTime = itemList.find('baseTime').string
        category = itemList.find('category').string
        fcstDate = itemList.find('fcstDate').string
        fcstTime = itemList.find('fcstTime').string
        fcstValue = itemList.find('fcstValue').string
        nx = itemList.find('nx').string
        ny = itemList.find('ny').string
        weatherList.append([baseDate, baseTime, category, fcstDate, fcstTime, fcstValue, nx, ny])

    df = pd.DataFrame(weatherList,
                      columns=['baseDate', 'baseTime', 'category', 'fcstDate', 'fcstTime', 'fcstValue', 'nx', 'ny'])

    df['ffcstValue'] = df['fcstValue'].astype('float')
    df_twoday = df[df['fcstDate'].isin([today, tomorrow])].groupby(['fcstDate', 'category']).mean(
        'fcstValue').reset_index()
    pv_towday = df_twoday.pivot(index=['fcstDate'], columns='category', values='ffcstValue').reset_index()
    pv_towday['temp_avg'] = pv_towday[['TMN', 'TMX']].mean(axis=1)
    pv_towday[['fcstDate', 'R06', 'REH', 'temp_avg']]

    returndic = {'today_rain': round(pv_towday[pv_towday['fcstDate'] == today].R06.values[0],1),
                 'today_humidity': round(pv_towday[pv_towday['fcstDate'] == today].REH.values[0],1),
                 'today_temp': round(pv_towday[pv_towday['fcstDate'] == today].temp_avg.values[0],1),
                 'today_ws': round(pv_towday[pv_towday['fcstDate'] == today].WSD.values[0],1),
                 'tomday_rain': round(pv_towday[pv_towday['fcstDate'] == tomorrow].R06.values[0],1),
                 'tomday_humidity': round(pv_towday[pv_towday['fcstDate'] == tomorrow].REH.values[0],1),
                 'tomday_temp': round(pv_towday[pv_towday['fcstDate'] == tomorrow].temp_avg.values[0],1),
                 'tomday_ws': round(pv_towday[pv_towday['fcstDate'] == tomorrow].WSD.values[0],1),
                 'address': address
                 }
    return returndic

get_7weather('서울특별시')

20201108
20201115
20201116
http://apis.data.go.kr/1360000/VilageFcstInfoService/getUltraSrtNcst?serviceKey=6TuB1szn0aNUEln7nrp6jgTQuv5WvoOgxRfPqdBtNPKTVIwbPV7SGmirrRyUIU95CP8oFZHn2f2Yr3zOWPiSdA%3D%3D&numOfRows=1000&pageNo=1&base_date=20201108&base_time=0200&nx=60&ny=127


KeyError: "None of [Index(['TMN', 'TMX'], dtype='object', name='category')] are in the [columns]"

In [ ]:
# 예측가능한 A형간염의 정보를 가져옴
def getTopDisea(city='서울특별시'):
    # Rsquare를 추출
    conn = pymysql.connect(host='multi-bigdata.cljkqcsbb9ok.ap-northeast-2.rds.amazonaws.com', port=3306, user='edu02',
                           passwd='edu02', db='edudb01', cursorclass=pymysql.cursors.DictCursor)
    try:
        cur = conn.cursor()
        sql = '''
            SELECT *
            FROM Rsquare_D
            WHERE 1 = 1
        '''
        cur.execute(sql)
        result = cur.fetchall()
    finally:
        conn.close()
    dfProduct = pd.DataFrame(result)

    # 날씨를 추출
    addWeather = pd.DataFrame(getweather(city), index=[0])

    # 날씨와 Rsquare를 병합
    alldf = pd.merge(dfProduct, addWeather, left_on=["city"], right_on=["address"], how="inner")

    # weeksting과 bin을 만듬
    today = datetime.datetime.today()
    tomorrow = (datetime.datetime.today() + datetime.timedelta(days=1))

    # 내일의 예상 감염자수
    alldf['xa2'] = alldf['xa'].apply(lambda x: x.replace('[', '').replace(']', '').split(','))
    alldf['b2'] = alldf['b'].apply(lambda x: x.replace('[', '').replace(']', '').split(','))

    # 내일의 예상 클릭율
    alldf['tomestV'] = alldf.apply(lambda x: round(float(x['xa2'][0]) * x['tomday_temp'] + float(x['xa2'][1]) * x['tomday_rain'] + float(x['xa2'][2]) * x['tomwk0'] + float(x['xa2'][3]) * x['tomwk1'] + float(x['b2'][0]), 1), axis=1)
    # 오늘의 예상 클릭율
    alldf['toestV'] = alldf.apply(lambda x: round(float(x['xa2'][0]) * x['today_temp'] + float(x['xa2'][1]) * x['today_rain'] + float(x['xa2'][2]) * x['towk0'] + float(x['xa2'][3]) * x['towk1'] + float(x['b2'][0]), 1), axis=1)

    alldf['incRate'] = alldf.apply(lambda x: round((x['tomestV'] - x['toestV']) / x['toestV'] * 100, 1), axis=1)

    returndf = alldf[['city', 'rsquare', 'response', 'toestV', 'tomestV', 'incRate']]

    #     top10 = lambda x: x.sort_values(by=['rsquare', 'incRate'], axis=0, ascending=False)[:6]
    #     resultdfbycity = returndf.groupby('city').apply(top10)

    resultdfbycity = returndf.sort_values(by=['rsquare', 'incRate'], axis=0, ascending=False)[:6]

    dicsum = resultdfbycity.to_dict('records')

    #     for x in dicsum:
    #         print(dicsum[x].values())

    return dicsum

In [ ]:
city='서울특별시'

# Rsquare를 추출
conn = pymysql.connect(host='multi-bigdata.cljkqcsbb9ok.ap-northeast-2.rds.amazonaws.com', port=3306, user='edu02',
                       passwd='edu02', db='edudb01', cursorclass=pymysql.cursors.DictCursor)
try:
    cur = conn.cursor()
    sql = '''
        SELECT *
        FROM Rsquare_D
        WHERE 1 = 1
    '''
    cur.execute(sql)
    result = cur.fetchall()
finally:
    conn.close()
dfDiseas = pd.DataFrame(result)

# 날씨를 추출
addWeather = pd.DataFrame(getweather(city), index=[0])


In [6]:
from bs4 import BeautifulSoup
import urllib.request as req
import pandas as pd
import datetime
import pymysql

def getweather(address):

    today = datetime.datetime.today().strftime('%Y%m%d')
    tomorrow = (datetime.datetime.today() + datetime.timedelta(days=1)).strftime('%Y%m%d')
    # print(today)
    # print(tomorrow)

    dicloc = {
        'address': ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '강원도', '충청북도',
                    '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주도'],
        'x': [60, 98, 89, 55, 58, 67, 102, 66, 60, 73, 69, 68, 63, 51, 89, 91, 52],
        'y': [127, 76, 90, 124, 74, 100, 84, 103, 120, 134, 107, 100, 89, 67, 91, 77, 38]}

    dfloc = pd.DataFrame(dicloc)
    xyloc = dfloc[dfloc['address'].isin([address])]

    # api = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getUltraSrtFcst?serviceKey=' #초단기예보
    api = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?serviceKey='  # 동네예보
    key = '6TuB1szn0aNUEln7nrp6jgTQuv5WvoOgxRfPqdBtNPKTVIwbPV7SGmirrRyUIU95CP8oFZHn2f2Yr3zOWPiSdA%3D%3D'
    numOfRows = '1000'
    pageNo = '1'
    base_date = today
    base_time = '0200'
    # print(xyloc.iloc[0][1])
    # print(xyloc.iloc[0][2])

    nx = xyloc.iloc[0][1].astype('str')
    ny = xyloc.iloc[0][2].astype('str')

    savename = 'tmpweather.xml'

    url = api + key + '&numOfRows=' + \
          numOfRows + '&pageNo=' + pageNo + '&base_date=' + base_date + '&base_time=' + base_time + '&nx=' + nx + '&ny=' + ny

    req.urlretrieve(url, savename)

    xml = open(savename, 'r', encoding='utf-8').read()
    soup = BeautifulSoup(xml, 'xml')

    # print(soup.find_all('item'))

    weatherList = []
    for itemList in soup.find_all('item'):
        baseDate = itemList.find('baseDate').string
        baseTime = itemList.find('baseTime').string
        category = itemList.find('category').string
        fcstDate = itemList.find('fcstDate').string
        fcstTime = itemList.find('fcstTime').string
        fcstValue = itemList.find('fcstValue').string
        nx = itemList.find('nx').string
        ny = itemList.find('ny').string
        weatherList.append([baseDate, baseTime, category, fcstDate, fcstTime, fcstValue, nx, ny])

    df = pd.DataFrame(weatherList,
                      columns=['baseDate', 'baseTime', 'category', 'fcstDate', 'fcstTime', 'fcstValue', 'nx', 'ny'])

    df['ffcstValue'] = df['fcstValue'].astype('float')
    df_twoday = df[df['fcstDate'].isin([today, tomorrow])].groupby(['fcstDate', 'category']).mean(
        'fcstValue').reset_index()
    pv_towday = df_twoday.pivot(index=['fcstDate'], columns='category', values='ffcstValue').reset_index()
    pv_towday['temp_avg'] = pv_towday[['TMN', 'TMX']].mean(axis=1)
    pv_towday[['fcstDate', 'R06', 'REH', 'temp_avg']]

    returndic = {'today_rain': round(pv_towday[pv_towday['fcstDate'] == today].R06.values[0],1),
                 'today_humidity': round(pv_towday[pv_towday['fcstDate'] == today].REH.values[0],1),
                 'today_temp': round(pv_towday[pv_towday['fcstDate'] == today].temp_avg.values[0],1),
                 'today_ws': round(pv_towday[pv_towday['fcstDate'] == today].WSD.values[0],1),
                 'tomday_rain': round(pv_towday[pv_towday['fcstDate'] == tomorrow].R06.values[0],1),
                 'tomday_humidity': round(pv_towday[pv_towday['fcstDate'] == tomorrow].REH.values[0],1),
                 'tomday_temp': round(pv_towday[pv_towday['fcstDate'] == tomorrow].temp_avg.values[0],1),
                 'tomday_ws': round(pv_towday[pv_towday['fcstDate'] == tomorrow].WSD.values[0],1),
                 'address': address
                 }
    return returndic

In [25]:
import pandas as pd
import datetime
import pymysql

# 예측가능한 5개도시 A형간염의 정보를 가져옴
# def getTopDisea():
city=["경기도", "서울특별시", "대전광역시", "세종특별자치시", "인천광역시", "부산광역시", "울산광역시", "광주광역시", "대구광역시"]
# Rsquare를 추출
conn = pymysql.connect(host='multi-bigdata.cljkqcsbb9ok.ap-northeast-2.rds.amazonaws.com', port=3306, user='edu02',
                       passwd='edu02', db='edudb01', cursorclass=pymysql.cursors.DictCursor)
try:
    cur = conn.cursor()
    sql = '''
        SELECT *
        FROM Rsquare_D
        WHERE 1 = 1
    '''
    cur.execute(sql)
    result = cur.fetchall()
finally:
    conn.close()
dfDiseas = pd.DataFrame(result)
display(dfDiseas)
addWeather = pd.DataFrame()
# 날씨를 추출
for add in city:
    addWeather = addWeather.append(pd.DataFrame(getweather(add), index=[0]), ignore_index=True)
display(addWeather)

,city,rsquare,response,xa,b,dvlist,grouprow,shiftrow
0,경기도,0.883241,A형간염,"[0.13655402077564108, 0.006178054357540655, -0...",[18.90272623],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,1
1,서울특별시,0.790832,A형간염,"[0.10955660619376668, 0.009009602830194186, -0...",[6.58585593],"['temp_avg', 'amount_of_rain', 'r_humidity']",5D,0
2,대전광역시,0.705759,A형간염,"[0.06154093875743628, 0.024004551196574957, -0...",[7.10412844],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,1
3,세종특별자치시,0.660183,A형간염,"[0.014237349292283364, 0.0019848172534662374, ...",[0.13081197],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,0
4,인천광역시,0.644040,A형간염,"[0.019013604292295528, -0.004616109660583029, ...",[-0.18211097],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,0
5,부산광역시,0.447245,A형간염,"[0.015792461617072824, -0.001883362763584258, ...",[-1.22883088],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,2
6,울산광역시,0.429278,A형간염,"[0.001736868812869053, 0.0012793802855548466, ...",[0.00055355],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,0
7,광주광역시,0.577538,A형간염,"[0.0035871075267225605, 0.0025628180198241056,...",[0.14727503],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,1
8,대구광역시,0.488989,A형간염,"[0.004984440351250199, 0.0012141953502713269, ...",[0.07561555],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,0


,today_rain,today_humidity,today_temp,today_ws,tomday_rain,tomday_humidity,tomday_temp,tomday_ws,address
0,0.0,78.3,12.0,1.2,0.0,78.1,14.0,1.2,경기도
1,0.0,70.0,13.5,1.5,0.0,69.4,14.5,1.5,서울특별시
2,0.0,70.8,13.0,0.9,0.0,75.6,15.0,0.9,대전광역시
3,0.0,75.8,11.0,0.8,0.0,79.4,14.5,0.9,세종특별자치시
4,0.0,80.8,11.5,1.0,0.0,78.1,14.5,1.0,인천광역시
5,0.0,66.7,15.0,1.3,0.0,76.2,17.5,1.2,부산광역시
6,0.0,68.3,13.5,2.4,0.0,76.9,15.5,2.2,울산광역시
7,0.0,61.7,14.0,1.1,0.1,81.2,15.5,1.2,광주광역시
8,0.0,67.5,13.0,1.5,0.0,77.5,14.5,1.1,대구광역시


In [26]:
# 질병과 Rsquare를 병합
alldf = pd.merge(dfDiseas, addWeather, left_on=["city"], right_on=["address"], how="inner")
display(alldf)

,city,rsquare,response,xa,b,dvlist,grouprow,shiftrow,today_rain,today_humidity,today_temp,today_ws,tomday_rain,tomday_humidity,tomday_temp,tomday_ws,address
0,경기도,0.883241,A형간염,"[0.13655402077564108, 0.006178054357540655, -0...",[18.90272623],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,1,0.0,78.3,12.0,1.2,0.0,78.1,14.0,1.2,경기도
1,서울특별시,0.790832,A형간염,"[0.10955660619376668, 0.009009602830194186, -0...",[6.58585593],"['temp_avg', 'amount_of_rain', 'r_humidity']",5D,0,0.0,70.0,13.5,1.5,0.0,69.4,14.5,1.5,서울특별시
2,대전광역시,0.705759,A형간염,"[0.06154093875743628, 0.024004551196574957, -0...",[7.10412844],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,1,0.0,70.8,13.0,0.9,0.0,75.6,15.0,0.9,대전광역시
3,세종특별자치시,0.660183,A형간염,"[0.014237349292283364, 0.0019848172534662374, ...",[0.13081197],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,0,0.0,75.8,11.0,0.8,0.0,79.4,14.5,0.9,세종특별자치시
4,인천광역시,0.644040,A형간염,"[0.019013604292295528, -0.004616109660583029, ...",[-0.18211097],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,0,0.0,80.8,11.5,1.0,0.0,78.1,14.5,1.0,인천광역시
5,부산광역시,0.447245,A형간염,"[0.015792461617072824, -0.001883362763584258, ...",[-1.22883088],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,2,0.0,66.7,15.0,1.3,0.0,76.2,17.5,1.2,부산광역시
6,울산광역시,0.429278,A형간염,"[0.001736868812869053, 0.0012793802855548466, ...",[0.00055355],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,0,0.0,68.3,13.5,2.4,0.0,76.9,15.5,2.2,울산광역시
7,광주광역시,0.577538,A형간염,"[0.0035871075267225605, 0.0025628180198241056,...",[0.14727503],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,1,0.0,61.7,14.0,1.1,0.1,81.2,15.5,1.2,광주광역시
8,대구광역시,0.488989,A형간염,"[0.004984440351250199, 0.0012141953502713269, ...",[0.07561555],"['temp_avg', 'amount_of_rain', 'r_humidity']",7D,0,0.0,67.5,13.0,1.5,0.0,77.5,14.5,1.1,대구광역시


In [34]:
# ['temp_avg', 'amount_of_rain', 'r_humidity']
# weeksting과 bin을 만듬
today = datetime.datetime.today()
tomorrow = (datetime.datetime.today() + datetime.timedelta(days=1))

# 내일의 예상 감염자수
alldf['xa2'] = alldf['xa'].apply(lambda x: x.replace('[', '').replace(']', '').split(','))
alldf['b2'] = alldf['b'].apply(lambda x: x.replace('[', '').replace(']', '').split(','))
# display(alldf)

# 내일의 예상 클릭율
alldf['tomestV'] = alldf.apply(lambda x: round(float(x['xa2'][0]) * x['tomday_temp'] + float(x['xa2'][1]) * x['tomday_rain'] + float(x['xa2'][2]) * x['tomday_humidity'] + float(x['b2'][0]), 1), axis=1)
# 오늘의 예상 클릭율
alldf['toestV'] = alldf.apply(lambda x: round(float(x['xa2'][0]) * x['today_temp'] + float(x['xa2'][1]) * x['today_rain'] + float(x['xa2'][2]) * x['today_humidity']  + float(x['b2'][0]), 1), axis=1)

# alldf['incRate'] = alldf.apply(lambda x: round((x['tomestV'] - x['toestV']) / x['toestV'] * 100, 1), axis=1)

alldf['SUMtomestV'] = alldf['tomestV'].sum()

alldf['piValue'] = alldf.apply(lambda x: round((x['tomestV']) / x['SUMtomestV'] * 100, 1), axis=1)

returndf = alldf[['city', 'rsquare', 'response', 'toestV', 'tomestV', 'piValue']]

returndic = {'paiCity': returndf['city'].tolist(),
             'PaiValue': returndf['city'].tolist()
             }




['경기도',
 '서울특별시',
 '대전광역시',
 '세종특별자치시',
 '인천광역시',
 '부산광역시',
 '울산광역시',
 '광주광역시',
 '대구광역시']

In [21]:
alldf['tomestV'].sum()

41.99999999999999

In [ ]:
    returndic = {'today_rain': round(pv_towday[pv_towday['fcstDate'] == today].R06.values[0],1),
                 'today_humidity': round(pv_towday[pv_towday['fcstDate'] == today].REH.values[0],1),
                 'today_temp': round(pv_towday[pv_towday['fcstDate'] == today].temp_avg.values[0],1),
                 'today_ws': round(pv_towday[pv_towday['fcstDate'] == today].WSD.values[0],1),
                 'tomday_rain': round(pv_towday[pv_towday['fcstDate'] == tomorrow].R06.values[0],1),
                 'tomday_humidity': round(pv_towday[pv_towday['fcstDate'] == tomorrow].REH.values[0],1),
                 'tomday_temp': round(pv_towday[pv_towday['fcstDate'] == tomorrow].temp_avg.values[0],1),
                 'tomday_ws': round(pv_towday[pv_towday['fcstDate'] == tomorrow].WSD.values[0],1),
                 'address': address
                 }

In [ ]:
city = '강원도'
# 날씨를 추출
addWeather = pd.DataFrame(getweather(city), index=[0])

In [45]:

# print(addWeather['today_temp'])
# 회귀식
x = [0]*22

x[0] = addWeather['today_temp']

if addWeather['today_rain'][0] > 1:
    x[1] = 1
else:
    x[2] = 1
x[3] = addWeather['today_ws']
x[4] = addWeather['today_humidity']
if addWeather['address'][0] == '강원도':
    x[5] = 1
elif addWeather['address'][0] == '경기도':
    x[6] = 1
elif addWeather['address'][0] == '경상남도':
    x[7] = 1
elif addWeather['address'][0] == '경상북도':
    x[8] = 1
elif addWeather['address'][0] == '광주광역시':
    x[9] = 1
elif addWeather['address'] [0]== '대구광역시':
    x[10] = 1
elif addWeather['address'][0] == '대전광역시':
    x[11] = 1
elif addWeather['address'][0] == '부산광역시':
    x[12] = 1
elif addWeather['address'][0] == '서울특별시':
    x[13] = 1
elif addWeather['address'][0] == '세종특별자치시':
    x[14] = 1
elif addWeather['address'][0] == '울산광역시':
    x[15] = 1
elif addWeather['address'][0] == '인천광역시':
    x[16] = 1
elif addWeather['address'][0] == '전라남도':
    x[17] = 1
elif addWeather['address'][0] == '전라북도':
    x[18] = 1
elif addWeather['address'][0] == '제주도':
    x[19] = 1
elif addWeather['address'][0] == '충청남도':
    x[20] = 1
elif addWeather['address'][0] == '충청북도':
    x[21] = 1

coef = [0.09551, -0.12152, 0.12152, -0.01190, 0.02075,
        -14.02573, 110.23470, -2.49813, 3.12397, -14.34688, 1.42490, -14.19731,
        -2.29574, 71.28671, -33.57688, -24.05782, -13.63816, -7.88631, -15.37370, -24.05272, -10.62914, -9.49175]

reg = [ ix * c for ix, c in zip(x, coef) ] + 32.9931

display(reg)
# reglist = {'reg':reg}



[0    1.05061
 Name: today_temp, dtype: float64,
 -0.0,
 0.12152,
 0   -0.0119
 Name: today_ws, dtype: float64,
 0    1.520975
 Name: today_humidity, dtype: float64,
 -14.02573,
 0.0,
 -0.0,
 0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0]